In [1]:
import pdfplumber
import re
import sys
import os
import json

In [2]:
if (sys.platform == 'linux'):
    pdf = pdfplumber.open("/home/adamson/projects/cs/ref/angus/16_Itikawa_J_Phys_Chem_Ref_Data_35_31_2006.pdf")

In [3]:
cropDim=[0 for i in range(len(pdf.pages))]

In [4]:
def p4FilenameGenerator():
    return "../../data/raw/ext/itikawa_n2/total"

In [5]:
def p5FilenameGenerator():
    return "../../data/raw/ext/itikawa_n2/elastic"

In [6]:
def p6FilenameGenerator():
    return "../../data/raw/ext/itikawa_n2/momentum"

In [7]:
def p7FilenameGenerator():
    return "../../data/raw/ext/itikawa_n2/rot_0_2"

In [8]:
def p8FilenameGenerator():
    return "../../data/raw/ext/itikawa_n2/vib_0_1"

In [9]:
def p10FilenameGenerator(i):
    return "../../data/raw/ext/itikawa_n2/tr_001_" + str(i + 2).zfill(3)

In [10]:
def p11FilenameGenerator(i):
    return "../../data/raw/ext/itikawa_n2/tr_001_" + str(i + 6).zfill(3)

In [11]:
def itikawa_table(ipage, filenameGenerator, fileHeader, locateTables=False):
    if (cropDim[ipage] != 0):
        pg = pdf.pages[ipage]
        for i in range(len(cropDim[ipage])):
            pgCropped = pg.crop(cropDim[ipage][i])
            if (locateTables):
                display(pgCropped.to_image().reset().draw_rects(pgCropped.chars))
            else:
                textSpaceDelim = pgCropped.extract_text().split("\n")
                textCommaDelim = fileHeader + "\n"
                for j in range(len(textSpaceDelim)):
                    textCommaDelim = textCommaDelim + (re.sub("\s+", ",", textSpaceDelim[j].strip())) + "\n"
                filename = filenameGenerator(i)
                exists = os.path.isfile(filename)
                if exists:
                    print("Skipping " + filename + ". It exists already.")
                else:
                    f = open(filename, "x")
                    f.write(textCommaDelim)
                    f.close()

In [12]:
#TODO: refactor the multiple calls to display(pgCropped...)
def itikawa_multicolumn_table(ipage, filenameGenerator, fileHeader, locateTables=False, omitRegexp=''):
    if (cropDim[ipage] != 0):
        pg = pdf.pages[ipage]
        filename = filenameGenerator()
        exists = os.path.isfile(filename)
        if exists:
            print("Skipping " + filename + ". It exists already.")
        else:
            if (locateTables==False):
                f = open(filename, "x")
                textCommaDelim = fileHeader + "\n"
                for i in range(len(cropDim[ipage])):
                    pgCropped = pg.crop(cropDim[ipage][i])
                    textSpaceDelim = pgCropped.extract_text().split("\n")
                    for j in range(len(textSpaceDelim)):
                        if (omitRegexp == '' or not(bool(re.search(omitRegexp, textSpaceDelim[j])))):
                            textCommaDelim = textCommaDelim + (re.sub("\s+", ",", textSpaceDelim[j].strip())) + "\n"
                f.write(textCommaDelim)
                f.close()
            else:
                for i in range(len(cropDim[ipage])):
                    pgCropped = pg.crop(cropDim[ipage][i])
                    display(pgCropped.to_image().reset().draw_rects(pgCropped.chars))


In [22]:
#TODO: refactor the multiple calls to display(pgCropped...)
def itikawa_multicolumn_table_json(ipage, filenameGenerator, fileHeader, locateTables=False, omitRegexp=''):
    if (cropDim[ipage] != 0):
        pg = pdf.pages[ipage]
        filename = filenameGenerator()
        exists = os.path.isfile(filename)
        #if exists:
        #    print("Skipping " + filename + ". It exists already.")
        #else:
        if (locateTables==False):
            #f = open(filename, "x")
            textCommaDelim = fileHeader + "\n"
            for i in range(len(cropDim[ipage])):
                pgCropped = pg.crop(cropDim[ipage][i])
                textSpaceDelim = pgCropped.extract_text().split("\n")
                for j in range(len(textSpaceDelim)):
                    if (omitRegexp == '' or not(bool(re.search(omitRegexp, textSpaceDelim[j])))):
                        textCommaDelim = textCommaDelim + (re.sub("\s+", ",", textSpaceDelim[j].strip())) + "\n"
            #f.write(textCommaDelim)
            #f.close()
            print(json.dumps(textCommaDelim))
        else:
            for i in range(len(cropDim[ipage])):
                pgCropped = pg.crop(cropDim[ipage][i])
                display(pgCropped.to_image().reset().draw_rects(pgCropped.chars))


In [23]:
cropDim[3] = [(300, 503, 390, 720),(400, 503, 470, 720),(480, 503, 560, 720)]

In [24]:
itikawa_multicolumn_table_json(3, p4FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False, omitRegexp='[0-9]a')

"Energy(eV),Cross Section(1E-16 cm^2)\n0.1,4.88\n0.12,5.13\n0.15,5.56\n0.17,5.85\n0.2,6.25\n0.25,6.84\n0.3,7.32\n0.35,7.72\n0.4,8.06\n0.45,8.33\n0.5,8.61\n0.6,8.96\n0.7,9.25\n0.8,9.48\n0.9,9.66\n1.0,9.85\n5.0,11.8\n6.0,11.4\n7.0,11.4\n8.0,11.5\n9.0,11.7\n10,12.0\n12,12.4\n15,13.2\n17,13.5\n20,13.7\n25,13.5\n30,13.0\n35,12.4\n40,12.0\n45,11.6\n50,11.3\n60,10.7\n70,10.2\n80,9.72\n90,9.30\n100,8.94\n120,8.33\n150,7.48\n170,7.02\n200,6.43\n250,5.66\n300,5.04\n350,4.54\n400,4.15\n450,3.82\n500,3.55\n600,3.14\n700,2.79\n800,2.55\n900,2.32\n1000,2.13\n"


In [110]:
cropDim[4] = [(300, 470, 420, 730),(430, 470, 550, 730)]

In [115]:
itikawa_multicolumn_table(4, p5FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False, omitRegexp='[0-9]a')

In [139]:
cropDim[5] = [(300, 105, 390, 333),(390, 105, 480, 333),(480, 105, 570, 333)]

In [141]:
itikawa_multicolumn_table(5, p6FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False, omitRegexp='[0-9]a')

In [153]:
cropDim[6] = [(50, 105, 170, 275),(180, 105, 300, 275)]

In [155]:
itikawa_multicolumn_table(6, p7FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False)

In [172]:
cropDim[7] = [(52, 541, 300, 721)]

In [174]:
itikawa_multicolumn_table(7, p8FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False, omitRegexp='[0-9]a')

In [23]:
cropDim[9] = [(120, 440, 200, 750),(210,440,290,750),(290,440,400,750),(400,440,500,750)]

In [58]:
itikawa_table(9, p10FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False)

Skipping ../../data/raw/ext/itikawa_n2/tr_001_002. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_003. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_004. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_005. It exists already.


In [59]:
cropDim[10] = [(40, 120, 120, 450),(130,120,210,450),(210,120,290,450),
               (310,120,400,450),(400,120,480,450),(480,120,560,450)]

In [60]:
itikawa_table(10, p11FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)")

Skipping ../../data/raw/ext/itikawa_n2/tr_001_006. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_007. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_008. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_009. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_010. It exists already.
Skipping ../../data/raw/ext/itikawa_n2/tr_001_011. It exists already.
